In [4]:
import pickle
from jupyter_utils import jupyter_setup
from audio_processing import FeatureSet
jupyter_setup()

C:\Michi\acoustic_covid_detection\python


In [23]:
with open(f"data/Coswara_processed/pickles/2023_02_11_breathing15mfcc_highres_corrected.pickle", "rb") as f:
    feature_set1 = pickle.load(f)

In [24]:
feature_set1.participants[0].recordings

{'breathing-deep': <audio_recording.AudioRecording at 0x2170af0ae80>}

In [25]:
with open(f"data/Coswara_processed/pickles/2023_02_11_cough15mfcc_highres_corrected.pickle", "rb") as f:
    feature_set2 = pickle.load(f)

In [26]:
feature_set2.participants[0].recordings

{'breathing-deep': <audio_recording.AudioRecording at 0x21708a21dc0>}

In [27]:
with open(f"data/Coswara_processed/pickles/2023_02_11_logmel_cough_fs22kHz.pickle", "rb") as f:
    feature_set3 = pickle.load(f)

In [28]:
feature_set3.participants[0].recordings

{'cough-heavy': <audio_recording.AudioRecording at 0x217020b93d0>}